In [ ]:
# -- select * from  dsc_dwd.dwd_wh_dsc_inventory_dtl_di 
# -- where src = 'scale'
# -- and data_source in ('scale_hpi', 'scale_michelin', 'scale_fas', 'scale_bose')
# -- and inc_day in ('20211124', '20211117', '20211110', '20211103', 
# -- '20211027', '20211020', '20211013','20211006')
# -- and usage_flag = '1'

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta
import re
import os
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore")

print(os.listdir('data_down'))

['inv_7.csv']


In [2]:
%%time
df = pd.read_csv('./data_down/inv_7.csv', sep='\001')
df.columns = [re.sub('\w+\.', '', i) for i in list(df.columns)]
df = df.dropna(how = 'all', axis =1)
time_cols = pd.Series(df.columns)[pd.Series(df.columns).str.lower().str.findall('date|time').apply(len)>0]
df[time_cols] = df[time_cols].apply(lambda x: x.str.slice(0,10))
df9 = df

Wall time: 20.5 s


In [56]:
def load_data(oucode):
    """
    load bose data;
    所有类型的qty都要加起来哦
    只选择有多个收货日期的货物
    """
    df = df9
    df = df[df['ou_code'].astype(str) == oucode]
    df ['recived_date'] = pd.to_datetime(df['recived_date'])
    df = df[['wms_company_name', 'wms_warehouse_id','sku_code', 'sku_name', 'sku_desc', 'location',\
        'lock_codes', 'on_hand_qty', 'in_transit_qty','allocated_qty', 'shelf_days', 
        'recived_date','usage_flag', 'inc_day']]
    # qty = pd.Series(df.columns).str.extract('(.+qty)').dropna(axis = 0)
    # df['qty'] = df[qty[0]].sum(axis = 1).round(2)
    df['qty'] = df['on_hand_qty']
    
    # bose_inv.drop_duplicates(subset = ['sku_code', 'recived_date', 'lock_codes', 'inc_day', 'qty'])
    # 没有重复的 目前看....aaa
    df = df.groupby(['recived_date', 'sku_code', 'lock_codes','inc_day', 'wms_warehouse_id'], dropna = False).agg(
    {
        'qty':sum,
        'location': set
    }
    ).sort_values(['sku_code', 'recived_date']).reset_index()
    # 只选择有多个收货日期的货物
    filter0 = df.groupby(['sku_code'])['recived_date'].agg(
    {
        set
    }
        ).reset_index()

    filter0 = pd.DataFrame(filter0[filter0['set'].apply(len)> 1]['sku_code'].drop_duplicates())
    bose_inv = filter0.merge(df, on = ['sku_code'], how = 'inner')\
        .sort_values(['recived_date','sku_code', 'inc_day'])
    
    return bose_inv

# bose_inv = load_data('BOSE_SH')

In [106]:
# =======================
# # 只选择多次出库的货物
# filter = bose.groupby(['recived_date', 'sku_code'])['inc_day'].agg({set}).reset_index()
# filter = filter[filter['set'].apply(len)> 1][['recived_date','sku_code']].drop_duplicates();filter
# bose_inv.head(4)
 

In [4]:
%%time
df0 = pd.DataFrame()
def snapshot():
    """
    pivot table. inc_day 快照 作为 cols
    添加标记.
    """
    global df0, bose_inv
    for i in tqdm(bose_inv['sku_code'].unique(), ascii= False, colour = 'green'):
        df_out = bose_inv[bose_inv['sku_code'] == i]\
            .pivot_table(columns=['inc_day'], index = 'recived_date', values=['qty']).reset_index()
        df_out['sku_code'] = i
        df0 = pd.concat([df0, df_out], axis = 0)
    try:
        df0.columns = df0.columns.get_level_values(level=1)
    except:
        pass
    df0.columns = list(df0.columns[0:8]) + ['received_date','sku']
    df0 = df0.sort_values(['sku', 'received_date'])
    df0['mark'] = 0
    # 这个吊东`西不知道为什么是反选的 
    df0['mark'] = df0['mark'].where(df0.iloc[:, 0:7].isna().all(axis = 1) == False, 'new')
    df0['mark'] = df0['mark'].where(~df0.iloc[:,7].isna() , 'clear')
    # fill na~
    df0 = df0.fillna(0)
    bose_inv = bose_inv.rename({'sku_code':'sku', 'recived_date':'received_date'}, axis = 1)\
        .reset_index(drop = True).drop(['inc_day', 'qty'], axis = 1)
    bose_inv = bose_inv.drop_duplicates(subset = ['sku', 'received_date', 'lock_codes'])
    df0 = df0.merge(bose_inv, on = ['sku','received_date'], how = 'left')

    # may lock
    df0['mark'] = df0['mark'].where(df0.iloc[:, 0:8].fillna(0).nunique(axis = 1) > 1, 'may_lock')
snapshot()

100%|██████████| 956/956 [01:06<00:00, 14.31it/s]


Wall time: 1min 9s


In [5]:
def err_part():
    """
    findout who are the naught peach.
    err 中干掉了 new 干掉了maylock 
    """
    df_err = df0[df0['mark'] != 'new']
    # 补充可能被锁的标记
    # df_err['mark'] = df_err['mark'].where(df_err.iloc[:, 0:8].fillna(0).nunique(axis = 1) > 1, 'may_lock')
    # 干掉了maylock
    df_err = df_err[df_err['mark'] != 'may_lock'].sort_values(['sku', 'received_date'])
    df_err['change'] = df_err.iloc[:,0:8].diff(axis = 1).sum(axis = 1)
    shift = df_err.groupby(['sku', 'wms_warehouse_id']).shift(1) 
    # q = []
    # for i in shift['mark']:
    #     if pd.isna(i):
    #         q.append('clear')
    #     elif i =='clear':
    #         q.append('clear')
    #     else:
    #         q.append('check')
    # shift['mark'] = q
    # shift['mark2'] = shift.iloc[:,0:8].diff(axis = 1).sum(axis = 1)
    shift = shift[['mark','change']]
    shift.columns = ['lag_mark', 'lag_change']
    shift['lag_mark'] = shift['lag_mark'].where(~shift['lag_mark'].isna(), 'clear')
    df_err = pd.concat([df_err, shift], axis = 1)
    # df_wrong = df_err[(df_err['lag_mark'] != 'clear') & (df_err['change'] != 0)]
    # df_good  = df_err[(df_err['lag_mark'] != 'clear') | (df_err['change'] != 0)]
    return df_err
df_err = err_part()

In [8]:
df_err.to_pickle('./data_up/df_err_bose.p')
df0.to_pickle('./data_up/df0_bose.p')
df_err_bose = pd.read_pickle('./data_up/df_err_bose.p')
df0_bose = pd.read_pickle('./data_up/df0_bose.p')
df0_bose = df0_bose[df0_bose['wms_warehouse_id'] == 'BOSE_SH']
df_err_bose = df_err_bose[df_err_bose['wms_warehouse_id'] == 'BOSE_SH']

In [58]:

def output(df_err, df0):
    dishes = list(df_err[(df_err['lag_mark'] != 'clear') \
        & (df_err['change'] < 0)
        & (df_err['change'] != df_err['lag_change'])]['sku'].unique())
    print(df0['lock_codes'].unique())
    return df0[df0['sku'].isin(dishes)]
def check(sku, df0):
    return df0[(df0['lock_codes'] == '1000') & (df0['sku'].isin(sku))].sort_values(['sku','received_date'])


In [81]:
# pd.DataFrame(
#         view.groupby('sku')['mark'].apply(list).astype(str).str.contains('may') # may lock?
#         ).reset_index()

In [83]:
view = output(df_err_bose,df0_bose)
# 1000全新 3000退货 4000破损产品-
def ou_level_lock_codes(lock_code_to_eliminate):
    """
    正则. lock_code "需要"被排除的, 依赖view表格. 
    """
    # output(df_err_bose,df0_bose)['sku'].unique()
    select_none_lock  = pd.DataFrame(
        view.groupby('sku')['mark'].apply(list).astype(str).str.contains('may') # may lock?
        ).reset_index()
    select_none_lock2 = pd.DataFrame(
        view.groupby('sku')['lock_codes'].apply(list).astype(str).str.contains(lock_code_to_eliminate)
        ).reset_index()
    # 去重    
    bose_err_list = set(select_none_lock[~select_none_lock['mark']]['sku'].unique())
    bose_err_list2 = list(select_none_lock2[~select_none_lock2['lock_codes']]['sku'].unique())
    bose_err_list = list(bose_err_list.intersection(bose_err_list2))
    bose_err_list = list(set(bose_err_list))
    
    return bose_err_list

bose_err_list = ou_level_lock_codes('(4000|3000)')
bose_definite_wrong = check(bose_err_list, df0_bose)
bose_definite_wrong\
    # .to_csv('./data_up/bose_fifo.csv', index = None, encoding = 'utf_8_sig')

['1000' '4000' '3000' '0001' '0100' '0004' '0005' '2000' '5000' '1100'
 '0010']


,20211006,20211013,20211020,20211027,20211103,20211110,20211117,20211124,received_date,sku,mark,lock_codes,wms_warehouse_id,location
640,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2021-07-26,326114-0130,clear,1000,BOSE_SH,{K-45-051-03}
641,42.0,36.0,12.0,12.0,12.0,11.0,11.0,10.0,2021-08-11,326114-0130,0,1000,BOSE_SH,"{K-43-064-02, K-44-069-02, K-43-061-02}"
642,20.0,20.0,10.0,8.0,8.0,0.0,0.0,0.0,2021-08-13,326114-0130,clear,1000,BOSE_SH,"{K-43-119-02, K-43-038-02}"
692,17.0,17.0,17.0,15.0,11.0,10.0,4.0,1.0,2021-01-27,328897-002S,0,1000,BOSE_SH,{K-45-118-03}
693,10.0,6.0,3.0,0.0,0.0,0.0,0.0,0.0,2021-06-08,328897-002S,clear,1000,BOSE_SH,{S-01-017-08}
1546,125.0,61.0,54.0,44.0,44.0,18.0,16.0,2.0,2021-06-24,638421-0110,0,1000,BOSE_SH,"{K-41-115-05, K-42-067-05, K-45-125-04, K-45-0..."
1547,4.0,4.0,4.0,4.0,4.0,4.0,1.0,1.0,2021-07-01,638421-0110,0,1000,BOSE_SH,{K-45-034-01}


=================================
# HP
=================================


In [231]:
%%time
# del df0 ,bose_inv
bose_inv = load_data('HPI_WH');bose_inv
df0 = pd.DataFrame()
snapshot()

# snapshot()
df_err = err_part()

100%|██████████| 280/280 [00:05<00:00, 47.86it/s]


Wall time: 6.42 s


In [209]:
# df0_hp['lock_codes'].unique()

In [232]:
df_err.to_pickle('./data_up/df_err_hpwh.p')
df0.to_pickle('./data_up/df0_hpwh.p')
df_err_hp = pd.read_pickle('./data_up/df_err_hpwh.p')
df0_hp = pd.read_pickle('./data_up/df0_hpwh.p')


In [233]:
view = output(df_err_hp, df0_hp)
err_list_hp = ou_level_lock_codes('(QH|27)')

In [239]:
def check(sku, df0):
    return df0[df0['sku'].isin(sku)]
hp_definite_wrong = check(err_list_hp, df0_hp)

# Z7Y79A 问一下
# 
hp_definite_wrong.head(6)

,20211006,20211013,20211020,20211027,20211103,20211110,20211117,20211124,received_date,sku,mark,lock_codes,wms_warehouse_id,location
377,20.0,19.0,19.0,17.0,17.0,9.0,8.0,8.0,2021-09-08,5RC89A#AB2,0,1002-34,HPI_WH,"{A-07-20-1, A-07-22-1, A-07-21-1, A-07-25-1, A..."
378,21.0,21.0,21.0,0.0,0.0,0.0,0.0,0.0,2021-09-22,5RC89A#AB2,clear,1001-34,HPI_WH,"{C-10-17-1, C-10-20-1, C-10-18-1, C-10-21-1, C..."
379,0.0,0.0,90.0,81.0,81.0,10.0,8.0,0.0,2021-10-18,5RC89A#AB2,clear,1001-34,HPI_WH,"{C-08-18-1, C-03-03-1, C-04-03-1, C-08-28-1, C..."
380,0.0,0.0,0.0,0.0,0.0,0.0,50.0,8.0,2021-11-15,5RC89A#AB2,0,1001-34,HPI_WH,"{C-08-02-1, C-08-01-1, C-07-04-1, C-07-06-1, C..."
442,16.0,16.0,16.0,16.0,13.0,12.0,10.0,6.0,2021-09-10,8GS43A#AB2,0,1002-34,HPI_WH,"{C-03-18-1, C-03-16-1, C-03-19-1, C-03-17-1}"
443,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2021-09-17,8GS43A#AB2,clear,1001-34,HPI_WH,{A-10-19-1}


In [163]:
# hp_definite_wrong[hp_definite_wrong['wms_warehouse_id']=='HPI_WH']

In [237]:
hp_definite_wrong.to_csv('./data_up/hpwh_fifo.csv', index = None, encoding = 'utf_8_sig')

In [245]:
# hp_definite_wrong[hp_definite_wrong['sku'] == '1VV21AA']

In [241]:
hp_definite_wrong.to_csv('./data_up/hpsh_fifo.csv', index = None, encoding = 'utf_8_sig')

=================================
# michelin
=================================


In [65]:
df0 = []
bose_inv = []

In [264]:
df9[df9['wms_warehouse_name'] == 'OE']

,wms_company_id,wms_company_name,wms_warehouse_id,wms_warehouse_name,lpn,sku_code,sku_name,sku_desc,product_status,batch_number,...,lock_codes,create_date,last_modified_date,data_source,manufactured_date,expiration_date,ou_code,usage_flag,inc_day,src
15559,Michelin,Michelin,OE,OE,NaN,028937_108,NaN,NaN,ONHAND_CN-1Y5,2123N0,...,ONHAND_CN-1Y5,2021-06-08,2021-07-04,scale_michelin,NaN,2024-06-01,MICHESHXCS,1,20211110,scale
15560,Michelin,Michelin,OE,OE,NaN,047665_101,NaN,NaN,ONHAND_CN-1Y5,2140N0,...,ONHAND_CN-1Y5,2021-10-10,2021-10-10,scale_michelin,NaN,2024-10-01,MICHESHXCS,1,20211110,scale
15561,Michelin,Michelin,OE,OE,NaN,047665_101,NaN,NaN,ONHAND_CN-1Y5,2141N0,...,ONHAND_CN-1Y5,2021-10-11,2021-10-14,scale_michelin,NaN,2024-10-01,MICHESHXCS,1,20211110,scale
15562,Michelin,Michelin,OE,OE,NaN,047665_101,NaN,NaN,ONHAND_CN-1Y5,2141N0,...,ONHAND_CN-1Y5,2021-10-10,2021-10-15,scale_michelin,NaN,2024-10-01,MICHESHXCS,1,20211110,scale
15563,Michelin,Michelin,OE,OE,NaN,047665_101,NaN,NaN,ONHAND_CN-1Y5,2142N0,...,ONHAND_CN-1Y5,2021-10-18,2021-10-21,scale_michelin,NaN,2024-10-01,MICHESHXCS,1,20211110,scale
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445068,Michelin,Michelin,OE,OE,NaN,996667_103,NaN,NaN,ONHAND_CN-17V,2136N0,...,ONHAND_CN-17V,2021-09-30,2021-10-04,scale_michelin,NaN,2024-09-04,MICHESHXCS,1,20211006,scale
445069,Michelin,Michelin,OE,OE,NaN,996667_103,NaN,NaN,ONHAND_CN-17V,2136N0,...,ONHAND_CN-17V,2021-10-03,2021-10-04,scale_michelin,NaN,2024-09-04,MICHESHXCS,1,20211006,scale
445070,Michelin,Michelin,OE,OE,NaN,996667_103,NaN,NaN,ONHAND_CN-17V,2136N0,...,ONHAND_CN-17V,2021-10-03,2021-10-06,scale_michelin,NaN,2024-09-04,MICHESHXCS,1,20211006,scale
445071,Michelin,Michelin,OE,OE,NaN,996667_103,NaN,NaN,ONHAND_CN-17V,2136N0,...,ONHAND_CN-17V,2021-10-05,2021-10-05,scale_michelin,NaN,2024-09-04,MICHESHXCS,1,20211006,scale


In [254]:
%%time
# del df0 ,bose_inv
bose_inv = load_data('RT');bose_inv
df0 = pd.DataFrame()
snapshot()

# snapshot()
df_err = err_part()

100%|██████████| 965/965 [00:22<00:00, 42.25it/s]


Wall time: 24.5 s


In [265]:
df_err.to_pickle('./data_up/df_err_michrt.p')
df0.to_pickle('./data_up/df0_michrt.p')
df_err_mich = pd.read_pickle('./data_up/df_err_michrt.p')
df0_mich = pd.read_pickle('./data_up/df0_michrt.p')


In [266]:
output(df0=df0_mich, df_err=df_err_mich).head(3)

,20211006,20211013,20211020,20211027,20211103,20211110,20211117,20211124,received_date,sku,mark,lock_codes,wms_warehouse_id,location
59,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2020-04-28,003789_101,may_lock,RETURN_IM-1IM,RT,{NC-GC-20}
60,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,2020-08-18,003789_101,clear,ONHAND_IM-1IM,RT,{E-25-17}
61,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2020-09-18,003789_101,clear,ONHAND_IM-1IM,RT,{E-25-17}


In [178]:
# df0_mich['lock_codes'].str.slice(0,7).unique()

In [267]:
view = output(df0=df0_mich, df_err=df_err_mich)
err_list_mich = ou_level_lock_codes('(BLOCKED_TH|RETURN)')

In [268]:
def check(sku, df0):
    return df0[df0['sku'].isin(sku)]

mich_definite_wrong = check(err_list_mich, df0 = df0_mich)

# MATMICHELIN00591

# 同一时间 received 货物为什么会增加
# mich dot date?

In [269]:
df0_mich[df0_mich['sku'] == '085753_111']

,20211006,20211013,20211020,20211027,20211103,20211110,20211117,20211124,received_date,sku,mark,lock_codes,wms_warehouse_id,location
698,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2021-07-25,085753_111,clear,Available,RT,{SHIP01}
699,160.0,172.0,162.0,158.0,149.0,145.0,129.0,129.0,2021-08-10,085753_111,0,Available,RT,{SCD-01-11}
700,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2021-09-29,085753_111,clear,ONHAND_TH-10C,RT,"{E-14-19, SHIP01}"


In [270]:
mich_definite_wrong[mich_definite_wrong['sku'] == '085753_111']

,20211006,20211013,20211020,20211027,20211103,20211110,20211117,20211124,received_date,sku,mark,lock_codes,wms_warehouse_id,location
698,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2021-07-25,085753_111,clear,Available,RT,{SHIP01}
699,160.0,172.0,162.0,158.0,149.0,145.0,129.0,129.0,2021-08-10,085753_111,0,Available,RT,{SCD-01-11}
700,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2021-09-29,085753_111,clear,ONHAND_TH-10C,RT,"{E-14-19, SHIP01}"


In [271]:
mich_definite_wrong.to_csv('./data_up/michrt_fifo.csv', index = None, encoding = 'utf_8_sig')

In [279]:
%%time
# del df0 ,bose_inv
bose_inv = load_data('OE');bose_inv
df0 = pd.DataFrame()
snapshot()
df_err = err_part()
view = output(df0=df0, df_err=df_err)
err_list_mich = ou_level_lock_codes('(BLOCKED_TH|RETURN)')
def check(sku, df0):
    return df0[df0['sku'].isin(sku)]

mich_definite_wrong = check(err_list_mich, df0 = df0)

mich_definite_wrong.head()

100%|██████████| 55/55 [00:01<00:00, 33.86it/s]


Wall time: 1.96 s


,20211006,20211013,20211020,20211027,20211103,20211110,20211117,20211124,received_date,sku,mark,lock_codes,wms_warehouse_id,location
397,482.0,482.0,145.0,0.0,0.0,0.0,0.0,0.0,2021-08-03,439132_101,clear,ONHAND_US-1AP,OE,{JGB-03-03}
398,908.0,908.0,908.0,243.0,0.0,0.0,0.0,0.0,2021-08-04,439132_101,clear,ONHAND_US-1AP,OE,{JGB-04-03}
399,988.0,988.0,988.0,988.0,0.0,0.0,0.0,0.0,2021-08-06,439132_101,clear,ONHAND_US-1AP,OE,{JGB-01-08}
400,653.0,653.0,652.0,652.0,490.0,187.0,0.0,0.0,2021-09-08,439132_101,clear,ONHAND_US-1AP,OE,"{JGA-04-13, JGA-04-15}"
401,1280.0,1280.0,1280.0,1279.0,1279.0,20.0,0.0,0.0,2021-09-09,439132_101,clear,ONHAND_US-1AP,OE,"{JGA-04-05, JGA-03-13}"


In [ ]:
mich_definite_wrong.to_csv('./data_up/michoe_fifo.csv', index = None, encoding = 'utf_8_sig')

==========================
# fas <br>
==========================

In [21]:
bose_inv = load_data('FAS') 
df0 = pd.DataFrame()
snapshot()

100%|██████████| 571/571 [00:16<00:00, 33.98it/s]


In [22]:
df_err = err_part()
view = output(df_err, df0)

In [53]:
bose_inv.head(3)

,sku,received_date,lock_codes,wms_warehouse_id,location
0,011373,2004-10-08,Available,FAS,{FS-74-A-007-PD}
8,011373,2004-11-11,Available,FAS,{FS-74-A-007-PD}
16,011373,2005-10-20,Available,FAS,{FS-74-A-007-PD}


In [49]:
err_list_hp = ou_level_lock_codes('(Avi)') 
bose_inv[bose_inv['sku'] == err_list_hp[0]]

,sku,received_date,lock_codes,wms_warehouse_id,location
23196,CRR218,2021-09-01,Available,FAS,"{FS-65-B-019-JN, FS-65-B-016-JN, FS-65-B-017-J..."
23348,CRR218,2021-09-03,Available,FAS,"{01-A-02-JN, FS-52-A-001-JN, FS-54-C-015-JN, F..."
23835,CRR218,2021-09-15,Available,FAS,"{FS-63-D-015-U, FS-62-D-015-U}"


In [54]:

def check(sku, df0):
    return df0[df0['sku'].isin(sku)]
hp_definite_wrong = check(err_list_hp, df0)

# hp_definite_wrong.head(6)

In [52]:
hp_definite_wrong.to_csv('./data_up/fas.csv', index = None, encoding = 'utf_8_sig')

==========================
# part2 <br>
==========================

select * from  dsc_dwd.dwd_wh_dsc_inventory_dtl_di 
where src = 'scale'
and  ou_code in(
'FERRECDXXS',
'APPLESHWHW',
'HUSQVSHMFS',
'COSTASHHTS',
'FUJIXSYXXS',
'ZEBRASHALS'
)
and inc_day in ('20211124', '20211117', '20211110', '20211103', 
'20211027', '20211020', '20211013','20211006')
and usage_flag = '1'

In [ ]:
%%time
df = pd.read_csv('./data_down/inv_7_2.csv', sep='\001')
df.columns = [re.sub('\w+\.', '', i) for i in list(df.columns)]
df = df.dropna(how = 'all', axis =1)
time_cols = pd.Series(df.columns)[pd.Series(df.columns).str.lower().str.findall('date|time').apply(len)>0]
df[time_cols] = df[time_cols].apply(lambda x: x.str.slice(0,10))
df9 = df

## Ferrero

In [90]:
bose_inv = load_data('FERRECDXXS') 
df0 = pd.DataFrame()
snapshot()
df_err = err_part()
view = output(df_err, df0)

100%|██████████| 80/80 [00:02<00:00, 38.34it/s]


['Unrestricted-FGAV' 'Unrestricted-DAMG' 'Blocked-FGAV' 'Blocked-BIOL']


In [104]:
err_list_hp = ou_level_lock_codes('Blocked') 
bose_inv[bose_inv['sku'] == err_list_hp[0]]

,sku,received_date,lock_codes,wms_warehouse_id,location
164,77221236,2021-09-24,Unrestricted-DAMG,CD_BTS,{G043801}
165,77221236,2021-09-24,Unrestricted-FGAV,CD_BTS,"{SHIP01, G032101}"
225,77221236,2021-09-30,Unrestricted-FGAV,CD_BTS,"{G010702, G011605}"
285,77221236,2021-10-07,Unrestricted-FGAV,CD_BTS,"{G013203, G014203}"
320,77221236,2021-10-12,Unrestricted-FGAV,CD_BTS,"{G020704, G021104, G020904, G021304, G020504, ..."
417,77221236,2021-10-17,Unrestricted-FGAV,CD_BTS,"{G010403, G010402}"
569,77221236,2021-10-22,Unrestricted-FGAV,CD_BTS,"{G022504, G022503, G023002, G022704, G022703, ..."
923,77221236,2021-11-05,Unrestricted-FGAV,CD_BTS,"{G011104, G010704, G012702, G011504, G011304, ..."
1147,77221236,2021-11-18,Unrestricted-FGAV,CD_BTS,"{G014302, G014703, G013104, G014303, G014503, ..."
1264,77221236,2021-11-28,Unrestricted-FGAV,CD_BTS,"{G012303, G015803, G013603, G015804, G015603, ..."


In [105]:
def check(sku, df0):
    return df0[df0['sku'].isin(sku)]
hp_definite_wrong = check(err_list_hp, df0)

# hp_definite_wrong.head(6)

In [107]:
hp_definite_wrong.to_csv('./data_up/ferrero_FERRECDXXS.csv', index = None, encoding = 'utf_8_sig')

## Apple sh

In [108]:
df.ou_code.unique()

array(['APPLESHWHW', 'COSTASHHTS', 'FERRECDXXS', 'FUJIXSYXXS',
       'HUSQVSHMFS', 'ZEBRASHALS'], dtype=object)

In [109]:
bose_inv = load_data('APPLESHWHW') 
df0 = pd.DataFrame()
snapshot()
df_err = err_part()
view = output(df_err, df0)

100%|██████████| 254/254 [00:08<00:00, 30.54it/s]


['XAVL-UNRESTRICTED-LC02AOI' 'XAVL-Unrestricted-LC01AOI'
 'XAVL-UNRESTRICTED-SOI' 'XAVL-Unrestricted-SOI' 'XHA-SOI'
 'XAVL-Unrestricted-LE01AOI' 'XAVL-Unrestricted-LC03AOI'
 'XAVL-UNRESTRICTED-LE02AOI' 'XAVL-Unrestricted-LE04AOI'
 'XAVL-Unrestricted-LE03AOI' 'Available' 'XAVL-Unrestricted-LC04AOI']


In [114]:
err_list_hp

['333-00075',
 '333-00038',
 '335S00506',
 '335S00437',
 '339S00667',
 '339S00668',
 '338S00578',
 '335S00515',
 'N-0003991',
 '337S00799']

In [113]:
err_list_hp = ou_level_lock_codes('ttt') 
# bose_inv[bose_inv['sku'] == err_list_hp[0]]

In [115]:
def check(sku, df0):
    return df0[df0['sku'].isin(sku)]
hp_definite_wrong = check(err_list_hp, df0)

# hp_definite_wrong.head(6)

In [119]:
# hp_definite_wrong[hp_definite_wrong['sku'] == '339S00668']

In [120]:
hp_definite_wrong.to_csv('./data_up/apple_sh_APPLESHWHW.csv', index = None, encoding = 'utf_8_sig')

## costa 

In [121]:
bose_inv = load_data('COSTASHHTS') 
df0 = pd.DataFrame()
snapshot()
df_err = err_part()
view = output(df_err, df0)

100%|██████████| 539/539 [00:27<00:00, 19.90it/s]


['Available' 'Held']


In [123]:
err_list_hp = ou_level_lock_codes('Held') 
df0[df0['sku'] == err_list_hp[0]]

,20211013,20211020,20211027,20211103,20211110,20211117,20211124,20211201,received_date,sku,mark,lock_codes,wms_warehouse_id,location
1264,70.0,65.0,67.0,67.0,62.0,60.0,57.0,56.0,2020-11-20,COSTA050372,0,Available,COSTA_SH,{D-06-15-01-A}
1265,17.0,17.0,14.0,14.0,14.0,14.0,14.0,14.0,2020-12-12,COSTA050372,0,Available,COSTA_SH,{D-06-17-01-A}


In [124]:
def check(sku, df0):
    return df0[df0['sku'].isin(sku)]
hp_definite_wrong = check(err_list_hp, df0)

# hp_definite_wrong.head(6)

In [125]:
hp_definite_wrong.to_csv('./data_up/costa_COSTASHHTS.csv', index = None, encoding = 'utf_8_sig')

## ZEBRASHALS

In [126]:
bose_inv = load_data('ZEBRASHALS') 
df0 = pd.DataFrame()
snapshot()
df_err = err_part()
view = output(df_err, df0)

100%|██████████| 363/363 [00:16<00:00, 22.32it/s]


['FIG1' 'REW2' 'HOLD1' 'OSP1']


In [127]:
err_list_hp = ou_level_lock_codes('HOLD1') 
df0[df0['sku'] == err_list_hp[0]]

,20211013,20211020,20211027,20211103,20211110,20211117,20211124,20211201,received_date,sku,mark,lock_codes,wms_warehouse_id,location
1457,10.0,10.0,2.0,2.0,2.0,2.0,2.0,2.0,2021-09-16,ZT61046-T290100Z,0,FIG1,DHL_SH2,"{BH-01-01, BG-03-01}"
1458,10.0,10.0,2.0,2.0,2.0,2.0,2.0,2.0,2021-09-16,ZT61046-T290100Z,0,REW2,DHL_SH2,{HH3QUTN}
1459,12.0,12.0,10.0,0.0,0.0,0.0,0.0,0.0,2021-09-17,ZT61046-T290100Z,clear,FIG1,DHL_SH2,{BB-39-01}
1460,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2021-10-12,ZT61046-T290100Z,clear,FIG1,DHL_SH2,{PECV1}
1461,0.0,12.0,12.0,7.0,0.0,0.0,0.0,0.0,2021-10-15,ZT61046-T290100Z,clear,FIG1,DHL_SH2,{BE-36-04}
1462,0.0,0.0,62.0,0.0,0.0,0.0,0.0,0.0,2021-10-25,ZT61046-T290100Z,clear,FIG1,DHL_SH2,{PECV1}
1463,0.0,0.0,0.0,62.0,0.0,0.0,0.0,0.0,2021-10-28,ZT61046-T290100Z,clear,FIG1,DHL_SH2,{STAG1}


In [130]:
def check(sku, df0):
    return df0[df0['sku'].isin(sku)]
hp_definite_wrong = check(err_list_hp, df0)

hp_definite_wrong.head(6)

,20211013,20211020,20211027,20211103,20211110,20211117,20211124,20211201,received_date,sku,mark,lock_codes,wms_warehouse_id,location
1328,35.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2021-09-13,ZT41142-T390000Z,clear,FIG1,DHL_SH2,"{BH-36-01, BA-28-01, BJ-05-04}"
1329,77.5,45.0,45.0,45.0,25.0,25.0,25.0,23.5,2021-09-16,ZT41142-T390000Z,0,FIG1,DHL_SH2,"{BJ-18-04, BJ-02-04, BE-26-01, BJ-20-01, BH-40..."
1330,77.5,45.0,45.0,45.0,25.0,25.0,25.0,23.5,2021-09-16,ZT41142-T390000Z,0,REW2,NaN,{ss8qutn}
1331,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2021-10-12,ZT41142-T390000Z,clear,FIG1,DHL_SH2,{PECV1}
1332,0.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2021-10-15,ZT41142-T390000Z,0,FIG1,DHL_SH2,{BG-01-01}
1333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2021-11-25,ZT41142-T390000Z,new,FIG1,DHL_SH2,{BC-05-01}


In [131]:
hp_definite_wrong.to_csv('./data_up/zebra_ZEBRASHALS.csv', index = None, encoding = 'utf_8_sig')